## Curve fitting example with Covalent

To use this notebook, it is recommended to run everything in a clean `conda` environment. The environment can be created by using the provided `environment.yml` file at the root level.

### Create environment

```bash
conda env create --file environment.yml
```


### Start Covalent

```bash
covalent start --ignore-migrations
```

To for more verbose logging, `covalent` can be started in `DEBUG` mode

```bash
covalent start -d --ignore-migrations
```

In [ ]:
import numpy as np
import covalent as ct
import matplotlib.pyplot as plt
from typing import List

### Raw code

```python
x = np.arange(10)
y = np.random.random(10)
```

### Fit
```python
fit = np.polyfit(x, y, 10)
xnew = np.linspace(x[0], x[-1], 50)
ynew = fit(xnew)
```


# plot
```python
fig, ax = plt.subplots()
plt.plot(x, y, 'o', xnew, ynew)
plt.xlim(x[0]-1, x[-1]+1)
```

### Functionalize the code (structured)

In [ ]:
def fit_xy(x, y, order: int):
    z = np.polyfit(x, y, order)
    return np.poly1d(z)

def plot_fit(x, y, xnew, fit, order):
    ynew = fit(xnew)
    fig, ax = plt.subplots()
    plt.plot(x, y, 'o')
    plt.plot(xnew, ynew, '-')
    plt.xlim([x[0]-1, x[-1]+1])
    plt.xlabel(r"x", fontsize=14)
    plt.ylabel(f"Interpolant", fontsize=14)
    plt.title(f"Curve fit order: {order}", fontsize=14)
    return ax

def cfit(x, y, order):
    xnew = np.linspace(x[0], x[-1], 50)
    fit = fit_xy(x=x, y=y, order=order)
    return plot_fit(x=x, y=y, xnew=xnew, fit=fit, order=order)

## Covalentify workflow

### Make `fit_xy` electron

In [ ]:
@ct.electron
def fit_xy(x, y, order):
   z = np.polyfit(x, y, order)
   return np.poly1d(z)

### Make an electron to plot the results

In [ ]:
@ct.electron
def plot_fit(x, y, xnew, fit, order):
   ynew = fit(xnew)
   fig, ax = plt.subplots()
   plt.plot(x, y, 'o', color='r', ms=10, label=r'Data')
   plt.plot(xnew, ynew, '--', color='k', label=f'{order}th order polynomial fit')
   plt.xlim([x[0]-1, x[-1]+1])
   plt.xlabel(r"x", fontsize=14)
   plt.ylabel(f"Interpolant", fontsize=14)
   plt.title(f"Curve fitting with Covalent", fontsize=14)
   plt.legend(fontsize=12)
   plt.tight_layout()
   return ax

### Create lattice for curve fitting (workflow)

In [ ]:
@ct.lattice
def cfit(x, y, order):
   xnew = np.linspace(x[0], x[-1], 100)
   fit = fit_xy(x=x, y=y, order=order)
   return plot_fit(x=x, y=y, xnew=xnew, fit=fit, order=order)

### Dispatch workflow to covalent server

* `N: size of the problem`
* `X => [0, N]`
* `Y => Randomly sampled between [0, 1] N times`
* `order: is degree of polynomial to use to fit the data, randomly chosen between [5, 10]`

In [ ]:
x = np.arange(10)
y = np.random.random(10)
dispatch_id = ct.dispatch(cfit)(x = x, y = y, order=np.random.randint(5, 10))
print(dispatch_id)

[Covalent Server UI](http://localhost:48008)

## Get the workflow result

In [ ]:
result = ct.get_result(dispatch_id=dispatch_id, wait=True)
print(result.result)

# Multiple dispatches

* Multiple curve fitting experiments
* Build the inputs as a `list`
* Dispatch each one to Covalent simultaneously

In [ ]:
inputs = [{"x": np.arange(10), "y": np.random.random(10), "order": np.random.randint(5, 10)} for _ in range(10)]

In [ ]:
dispatches = [ct.dispatch(cfit)(input["x"], input["y"], order=input["order"]) for input in inputs]

In [ ]:
results = [ct.get_result(dispatch_id, wait=True).result for dispatch_id in dispatches]

# Sublattices

* Instead of create a multiple dispatches of the same workflow, it would be convenient to do a single dispatch of a larger workflow that invokes the original workflow
* Mark the first workflow as a `electron`
* Create a larger workflow that takes the space of parameters as an input
* Dispatch the new workflow

In [ ]:
cfit_sublattice = ct.electron(cfit)

In [ ]:
@ct.lattice
def multiple_curve_fits(inputs: List):
    all_results = []
    for input in inputs:
        result = cfit_sublattice(x=input["x"], y = input["y"], order=input["order"])
        all_results.append(result)
    return all_results

### Dispatch the larger workflow

In [ ]:
inputs = [{"x": np.arange(10), "y": np.random.random(10), "order": np.random.randint(5, 10)} for _ in range(5)]
dispatch_id = ct.dispatch(multiple_curve_fits)(inputs)

In [ ]:
results = ct.get_result(dispatch_id=dispatch_id, wait=True)

In [ ]:
print(results.result)

### Result object

Stores all the metadata and necessary information about the workflow as a Python object. Users can use the result object API to query the status of running and completed workflows at a very granular level

In [ ]:
x = np.arange(10)
y = np.random.random(10)
dispatch_id = ct.dispatch(cfit)(x = x, y = y, order = 10)
result = ct.get_result(dispatch_id, wait=True)

### Workflow status

In [ ]:
print(result.status)

### Retrive results of all nodes in the workflow

In [ ]:
result.get_all_node_results()

### Workflow execution time

In [ ]:
duration = result.end_time - result.start_time
print(duration.total_seconds())

### Query individual node outputs/metadata

In [ ]:
result.get_node_result(0)